# CIFAR-10 이미지 생성하기

## Step 1. 작업환경 구성하기

![디렉토리 만들기](https://user-images.githubusercontent.com/116326867/208328548-4412d3f9-a82d-4943-940d-40adeb176ccd.png)

In [1]:
# 필요한 모둘 import
import os
import glob
import time

import PIL
import imageio
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline

## Step 2. 데이터셋 구성하기

In [3]:
cifar10 = tf.keras.datasets.cifar10

(train_x, _), (test_x, _) = cifar10.load_data()

train_x.shape

(50000, 32, 32, 3)

## Step 3. 생성자 모델 구현하기